# AI Action Plan Comments Report

BMZA | 05-13-2025

This analysis examines public comments submitted in response to the National AI Action Plan, with a specific focus on those referencing or relevant to the U.S. Department of Commerce. The project leverages the AI-enhanced dataset provided by IFP[^2], which uses Claude (claude-3-7-sonnet-latest) to topic cluster, extract recommendations, and summarize across thousands of public submissions. By filtering and analyzing content related to the Department of Commerce, this work seeks to identify key public concerns, priorities, and recommendations regarding the Department’s role in AI governance, research, and regulation.

[^2]: [IFP AI Action Plan Database](aiactionplan.org)

## Background and Context

In February 2025, the White House Office of Science and Technology Policy (OSTP) issued a Request for Information (RFI) to solicit public input on the development of a national AI Action Plan.[^1] This initiative was launched under President Trump’s Executive Order on Removing Barriers to American Leadership in AI, aiming to gather perspectives from a wide range of stakeholders—including academia, industry, civil society, and various levels of government—on AI policy priorities. Over a seven-week period, more than 10,000 comments were submitted, covering topics such as semiconductor manufacturing, supply chain resilience, AI model development, workforce training, and scientific research. The comment period closed on March 15, 2025, and the full catalog of submissions was published by OSTP on April 24, 2025. [^2]

The Department of Commerce plays a pivotal role in the federal AI ecosystem through its sub-agencies, including the National Institute of Standards and Technology (NIST) and the U.S. Patent and Trademark Office (USPTO). These agencies are central to the development of technical standards, intellectual property frameworks, and innovation policy related to AI. Analyzing public feedback directed toward or implicating the Department of Commerce provides valuable insights into stakeholder expectations and concerns, thereby informing the Department’s ongoing efforts in AI governance and policy formulation.

[^1]: [Request for Information on the Development of an Artificial Intelligence (AI) Action Plan](https://www.federalregister.gov/documents/2025/02/06/2025-02305/request-for-information-on-the-development-of-an-artificial-intelligence-ai-action-plan)
[^2]: [The White House: American Public Submits Over 10,000 Comments on White House’s AI Action Plan](https://www.whitehouse.gov/articles/2025/04/american-public-submits-over-10000-comments-on-white-houses-ai-action-plan/)


In [5]:
import pandas as pd
import nbconvert

# Importing data from aiactionplan.org
data = pd.read_csv('recommendations.csv')

# Extracting Commerce assignees
assignees = [
    "All",
    "NIST",
    "Department of Commerce",
    "AISI",
    "Bureau of Industry and Security (BIS)",
    "USPTO",
    "National Semiconductor Technology Center",
    "NOAA",
    "Census Bureau",
    "USTR",
    "NAIC",
    "CHIPS Program Office",
    "Manufacturing USA",
    "PTO",
    "SelectUSA",
    "Patent and Trial Appeal Board",
    "Foreign Commercial Service",
    "NCI",
    "EDA",
    "CHIPS Manufacturing USA",
    "End-User Review Committee",
    "Bureau of Economic Analysis"
]
commerce_recommendations = data[data['Assignee'].isin(assignees)]
commerce_recommendations.to_excel('commerce_recommendations_ai_action_plan_comments.xlsx')

## Breakdown by Organization Type

In [9]:
# ---- Step 1: Imports ----
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import display, HTML

# ---- Step 2: Set the correct renderer ----
# This helps plotly embed charts correctly in nbconvert HTML
pio.renderers.default = 'notebook_connected'

# ---- Step 3: Create example data ----
# Group and count by OrgType
org_counts = commerce_recommendations['OrgType'].value_counts().reset_index()
org_counts.columns = ['OrgType', 'Count']

# Plot using Plotly Express
fig = px.bar(
    org_counts,
    x='OrgType',
    y='Count',
    title='Count of Recommendations by Organization Type',
    labels={'OrgType': 'Organization Type', 'Count': 'Number of Recommendations'},
    text='Count'
)

fig.update_traces(textposition='outside')
fig.update_layout(xaxis_tickangle=-45)


display(fig)


The majority of these assigned comments come from industry associations. One thing i want to add here is the comparison of overall comments vs department comments.

## Recommendations by topic, stacked by bureau

In [7]:
exploded_df = commerce_recommendations.copy()
exploded_df['Topic'] = exploded_df['Topic'].str.split(',')

# Remove leading/trailing whitespace from topics
exploded_df = exploded_df.explode('Topic')
exploded_df['Topic'] = exploded_df['Topic'].str.strip()

# Step 2: Group by cleaned Topic and Assignee, then count
grouped = exploded_df.groupby(['Topic', 'Assignee']).size().reset_index(name='Count')

# Step 3: Pivot the table to get one row per Topic
topic_wide = grouped.pivot(index='Topic', columns='Assignee', values='Count').fillna(0).astype(int).reset_index()

# Optional: sort by total count across all assignees
topic_wide['Total'] = topic_wide.drop(columns='Topic').sum(axis=1)
topic_wide = topic_wide.sort_values(by='Total', ascending=False).drop(columns='Total')

topic_wide

df = topic_wide


# Combine AISI and National Semiconductor Technology Center into NIST
df['NIST'] = df['NIST'] + df['AISI'] + df['National Semiconductor Technology Center']

# Combine PTO and Patent and Trial Appeal Board into USPTO
df['USPTO'] = df['USPTO'] + df['PTO'] + df['Patent and Trial Appeal Board']

# Drop merged columns
df = df.drop(columns=['AISI', 'National Semiconductor Technology Center', 'PTO', 'Patent and Trial Appeal Board'])

# Melt to long format
df_long = df.melt(id_vars='Topic', var_name='Assignee', value_name='Count')
df_long = df_long[df_long['Count'] > 0]
# Calculate total counts for each topic
totals = df_long.groupby('Topic')['Count'].sum().reset_index()
# Reorder Topic by total count (largest to smallest)
topic_order = totals.sort_values('Count', ascending=False)['Topic']
df_long['Topic'] = pd.Categorical(df_long['Topic'], categories=topic_order, ordered=True)

# Create stacked bar chart
fig = px.bar(
    df_long,
    x='Topic',
    y='Count',
    color='Assignee',
    title='Recommendation Count by Topic (Stacked by Assignee)',
    labels={'Count': 'Number of Recommendations'},
    color_discrete_sequence=px.colors.qualitative.G10
)

fig.update_layout(barmode='stack', xaxis_tickangle=-45)


# Add annotations
for i, row in totals.iterrows():
    fig.add_annotation(
        x=row['Topic'],
        y=row['Count'],
        text=str(row['Count']),
        showarrow=False,
        yshift=10,
        font=dict(size=12, color='black')
    )

display(fig)


Most of the recommendations are assigned to NIST and have to do with standards and regulation.